In [ ]:
1. Handling Missing Values

In [4]:
import gdown
import pandas as pd
import os
file_id = "1sQJs53Qk3v35YnX40UfqWrSWQIcatykS"
url = f"https://drive.google.com/uc?id={file_id}"
output = "US_Accidents.csv"
if not os.path.exists(output):
    gdown.download(url, output, quiet=False)
    print("Downloaded from Google Drive")
else:
    print("File already exists locally")
    
df = pd.read_csv(output)

File already exists locally


C:\Users\polep\AppData\Local\Temp\ipykernel_16148\1160782198.py:13: DtypeWarning: Columns (37,38,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(output)


In [5]:
# Check total missing values in each column
missing_values = df.isnull().sum().sort_values(ascending=False)
print("Missing values per column:\n", missing_values)

# Percentage of missing values
missing_percent = (df.isnull().sum() / len(df)) * 100
print("\nPercentage missing:\n", missing_percent.sort_values(ascending=False))


Missing values per column:
 End_Lat                  713311
End_Lng                  713311
Precipitation(in)        465433
Wind_Chill(F)            454629
Wind_Speed(mph)           94991
Visibility(mi)            14305
Weather_Condition         13560
Humidity(%)               11895
Temperature(F)            10907
Pressure(in)               8590
Wind_Direction             8163
Weather_Timestamp          6534
Street                     1614
Nautical_Twilight          1437
Civil_Twilight             1437
Sunrise_Sunset             1437
Astronomical_Twilight      1437
Airport_Code                677
Timezone                    225
Zipcode                      76
City                         22
Turning_Loop                  1
Traffic_Signal                1
Traffic_Calming               1
Stop                          1
Junction                      0
Severity                      0
Start_Time                    0
End_Time                      0
Station                       0
Roundabout  

In [ ]:
2. Dropping and Imputing Columns/Rows with Missing Data

In [6]:
# Define threshold: drop columns with >30% missing
threshold = 0.3 * len(df)
df = df.dropna(thresh=threshold, axis=1)

print("Columns after dropping high-missing ones:\n", df.columns)


Columns after dropping high-missing ones:
 Index(['ID', 'Source', 'Severity', 'Start_Time', 'End_Time', 'Start_Lat',
       'Start_Lng', 'Distance(mi)', 'Description', 'Street', 'City', 'County',
       'State', 'Zipcode', 'Country', 'Timezone', 'Airport_Code',
       'Weather_Timestamp', 'Temperature(F)', 'Wind_Chill(F)', 'Humidity(%)',
       'Pressure(in)', 'Visibility(mi)', 'Wind_Direction', 'Wind_Speed(mph)',
       'Precipitation(in)', 'Weather_Condition', 'Amenity', 'Bump', 'Crossing',
       'Give_Way', 'Junction', 'No_Exit', 'Railway', 'Roundabout', 'Station',
       'Stop', 'Traffic_Calming', 'Traffic_Signal', 'Turning_Loop',
       'Sunrise_Sunset', 'Civil_Twilight', 'Nautical_Twilight',
       'Astronomical_Twilight'],
      dtype='object')


In [7]:
# Drop rows where Start_Time, Start_Lat, or Start_Lng is missing
df = df.dropna(subset=["Start_Time", "Start_Lat", "Start_Lng"])

print("Shape after dropping critical rows:", df.shape)


Shape after dropping critical rows: (713311, 44)


In [8]:
numerical_cols = ["Temperature(F)", "Humidity(%)", "Pressure(in)", "Visibility(mi)", "Wind_Speed(mph)"]

for col in numerical_cols:                                            # impute numerical columns
    df[col].fillna(df[col].median(), inplace=True)


In [9]:
categorical_cols = ["Weather_Condition", "Wind_Direction", "Sunrise_Sunset"]   # impute categorical values

for col in categorical_cols:
    df[col].fillna(df[col].mode()[0], inplace=True)  # fill with most common value


In [10]:
print("Missing values after dropping/imputing:\n", df.isnull().sum())


Missing values after dropping/imputing:
 ID                            0
Source                        0
Severity                      0
Start_Time                    0
End_Time                      0
Start_Lat                     0
Start_Lng                     0
Distance(mi)                  0
Description                   0
Street                     1614
City                         22
County                        0
State                         0
Zipcode                      76
Country                       0
Timezone                    225
Airport_Code                677
Weather_Timestamp          6534
Temperature(F)                0
Wind_Chill(F)            454629
Humidity(%)                   0
Pressure(in)                  0
Visibility(mi)                0
Wind_Direction                0
Wind_Speed(mph)               0
Precipitation(in)        465433
Weather_Condition             0
Amenity                       0
Bump                          0
Crossing                      0

In [11]:
# Drop columns with too many missing values
df.drop(columns=["Wind_Chill(F)", "Precipitation(in)", "Weather_Timestamp"], inplace=True)

# Fill categorical columns with mode or "Unknown"
df['Street'].fillna("Unknown", inplace=True)
df['City'].fillna(df['City'].mode()[0], inplace=True)
df['Zipcode'].fillna(df['Zipcode'].mode()[0], inplace=True)
df['Timezone'].fillna(df['Timezone'].mode()[0], inplace=True)
df['Airport_Code'].fillna("Unknown", inplace=True)
df['Stop'].fillna(df['Stop'].mode()[0], inplace=True)
df['Traffic_Calming'].fillna(df['Traffic_Calming'].mode()[0], inplace=True)
df['Traffic_Signal'].fillna(df['Traffic_Signal'].mode()[0], inplace=True)
df['Turning_Loop'].fillna(df['Turning_Loop'].mode()[0], inplace=True)

# Fill twilight columns with mode
twilight_cols = ["Civil_Twilight", "Nautical_Twilight", "Astronomical_Twilight"]
for col in twilight_cols:
    df[col].fillna(df[col].mode()[0], inplace=True)

# Verify
print("Missing values after final imputation:\n", df.isnull().sum())


Missing values after final imputation:
 ID                       0
Source                   0
Severity                 0
Start_Time               0
End_Time                 0
Start_Lat                0
Start_Lng                0
Distance(mi)             0
Description              0
Street                   0
City                     0
County                   0
State                    0
Zipcode                  0
Country                  0
Timezone                 0
Airport_Code             0
Temperature(F)           0
Humidity(%)              0
Pressure(in)             0
Visibility(mi)           0
Wind_Direction           0
Wind_Speed(mph)          0
Weather_Condition        0
Amenity                  0
Bump                     0
Crossing                 0
Give_Way                 0
Junction                 0
No_Exit                  0
Railway                  0
Roundabout               0
Station                  0
Stop                     0
Traffic_Calming          0
Traffic_Signal 

In [12]:
import pandas as pd
import numpy as np    # just taken an example to show dropping of colums with 50 % of volumes

# Example DataFrame
data = {
    'A': [1, np.nan, 3, 4, 5],
    'B': [np.nan, np.nan, np.nan, 4, 5],
    'C': [1, 2, 3, np.nan, 5]
}
df = pd.DataFrame(data)
print("Original DataFrame:\n", df)

# Drop rows with ANY missing values
df_drop_rows = df.dropna(axis=0)
print("\nAfter dropping rows with any missing values:\n", df_drop_rows)

# Drop columns with ANY missing values
df_drop_cols = df.dropna(axis=1)
print("\nAfter dropping columns with any missing values:\n", df_drop_cols)

# Drop columns with more than a threshold (e.g., 50%) missing values
threshold = 0.5 * len(df)
df_drop_threshold = df.dropna(axis=1, thresh=threshold)
print("\nAfter dropping columns with >50% missing values:\n", df_drop_threshold)

Original DataFrame:
      A    B    C
0  1.0  NaN  1.0
1  NaN  NaN  2.0
2  3.0  NaN  3.0
3  4.0  4.0  NaN
4  5.0  5.0  5.0

After dropping rows with any missing values:
      A    B    C
4  5.0  5.0  5.0

After dropping columns with any missing values:
 Empty DataFrame
Columns: []
Index: [0, 1, 2, 3, 4]

After dropping columns with >50% missing values:
      A    C
0  1.0  1.0
1  NaN  2.0
2  3.0  3.0
3  4.0  NaN
4  5.0  5.0


In [ ]:
Imputing (Filling) Missing Values

In [ ]:
a) Fill with a Constant (e.g., 0, "unknown")

In [13]:
df_constant = df.fillna(0)
print("\nFill missing values with 0:\n", df_constant)


Fill missing values with 0:
      A    B    C
0  1.0  0.0  1.0
1  0.0  0.0  2.0
2  3.0  0.0  3.0
3  4.0  4.0  0.0
4  5.0  5.0  5.0


In [ ]:
b) Fill with Mean / Median / Mode

In [14]:
# Fill numerical columns with mean
df_mean = df.fillna(df.mean(numeric_only=True))
print("\nFill numerical columns with mean:\n", df_mean)

# Fill numerical columns with median
df_median = df.fillna(df.median(numeric_only=True))
print("\nFill numerical columns with median:\n", df_median)

# Fill categorical columns with mode
df_mode = df.apply(lambda x: x.fillna(x.mode()[0]) if x.dtype=='O' else x)
print("\nFill categorical columns with mode:\n", df_mode)


Fill numerical columns with mean:
       A    B     C
0  1.00  4.5  1.00
1  3.25  4.5  2.00
2  3.00  4.5  3.00
3  4.00  4.0  2.75
4  5.00  5.0  5.00

Fill numerical columns with median:
      A    B    C
0  1.0  4.5  1.0
1  3.5  4.5  2.0
2  3.0  4.5  3.0
3  4.0  4.0  2.5
4  5.0  5.0  5.0

Fill categorical columns with mode:
      A    B    C
0  1.0  NaN  1.0
1  NaN  NaN  2.0
2  3.0  NaN  3.0
3  4.0  4.0  NaN
4  5.0  5.0  5.0


In [ ]:
c) Forward Fill / Backward Fill (for time series or panel data)

In [15]:
df_ffill = df.fillna(method='ffill')
print("\nForward fill missing values:\n", df_ffill)

df_bfill = df.fillna(method='bfill')
print("\nBackward fill missing values:\n", df_bfill)


Forward fill missing values:
      A    B    C
0  1.0  NaN  1.0
1  1.0  NaN  2.0
2  3.0  NaN  3.0
3  4.0  4.0  3.0
4  5.0  5.0  5.0

Backward fill missing values:
      A    B    C
0  1.0  4.0  1.0
1  3.0  4.0  2.0
2  3.0  4.0  3.0
3  4.0  4.0  5.0
4  5.0  5.0  5.0


In [ ]:
d) Advanced: KNN Imputer / Iterative Imputer (for large projects)

In [16]:
from sklearn.impute import KNNImputer

knn_imputer = KNNImputer(n_neighbors=2)
df_knn = pd.DataFrame(knn_imputer.fit_transform(df), columns=df.columns)
print("\nKNN Imputed DataFrame:\n", df_knn)


KNN Imputed DataFrame:
      A    B    C
0  1.0  4.5  1.0
1  2.0  4.5  2.0
2  3.0  4.5  3.0
3  4.0  4.0  4.0
4  5.0  5.0  5.0


In [ ]:
3. Choosing the Best Approach
Guidelines:

Drop columns if lots of information is missing (>30–50%), but consider domain importance.

Impute when missingness is moderate, using domain knowledge to choose the technique.

Use mean/median for numeric, mode for categorical, and advanced methods for critical variables.

In [17]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer

# Create a dummy dataset with missing values
data = {
    'Age': [25, np.nan, 30, 22, 40, np.nan, 28],
    'Salary': [50000, 60000, np.nan, 52000, 58000, 62000, np.nan],
    'City': ['New York', 'Los Angeles', 'New York', np.nan, 'Chicago', 'Chicago', 'Los Angeles'],
    'Purchased': ['Yes', 'No', np.nan, 'No', 'Yes', 'Yes', 'No']
}

df_dummy = pd.DataFrame(data)
print("Original Dummy Dataset with Missing Values:")
print(df_dummy)

# Assignment Tasks (interns should attempt):
# 1. Drop columns or rows with excessive missing values.
# 2. Impute missing numerical columns (Age, Salary) with mean or median.
# 3. Impute missing categorical columns (City, Purchased) with mode.
# 4. Optionally, apply KNN imputation for numerical columns.

# Example imputation with mean and mode:
df_imputed = df_dummy.copy()
df_imputed['Age'] = df_imputed['Age'].fillna(df_imputed['Age'].mean())
df_imputed['Salary'] = df_imputed['Salary'].fillna(df_imputed['Salary'].median())
df_imputed['City'] = df_imputed['City'].fillna(df_imputed['City'].mode()[0])
df_imputed['Purchased'] = df_imputed['Purchased'].fillna(df_imputed['Purchased'].mode()[0])

print("\nDummy Dataset After Imputation:")
print(df_imputed)

Original Dummy Dataset with Missing Values:
    Age   Salary         City Purchased
0  25.0  50000.0     New York       Yes
1   NaN  60000.0  Los Angeles        No
2  30.0      NaN     New York       NaN
3  22.0  52000.0          NaN        No
4  40.0  58000.0      Chicago       Yes
5   NaN  62000.0      Chicago       Yes
6  28.0      NaN  Los Angeles        No

Dummy Dataset After Imputation:
    Age   Salary         City Purchased
0  25.0  50000.0     New York       Yes
1  29.0  60000.0  Los Angeles        No
2  30.0  58000.0     New York        No
3  22.0  52000.0      Chicago        No
4  40.0  58000.0      Chicago       Yes
5  29.0  62000.0      Chicago       Yes
6  28.0  58000.0  Los Angeles        No
